### 3. Exploratory Data Analysis And Data Preprocessing :
Pre-processing refers to the transformations applied to the data before feeding it to the algorithm. Data Preprocessing Includes Data Collection , Cleaning ,Conversion of features , Imputing missing data ,features standardizations , feature scaling and identify new potential features.

#### Source of Data Set :
   For this article, I have used a Data From (Need to be added ) data set from You can download the final training and testing data set from here:           
                
Lets get started by importing important packages and the data set.

In [1]:
%pylab inline 
import pandas as pd
import numpy as np

from scipy import stats, integrate
import matplotlib.pyplot as plt

import seaborn as sns
# special matplotlib argument for improved plots
from matplotlib import rcParams
import statsmodels.api as sm
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis




from sklearn.tree import DecisionTreeClassifier
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.utils.extmath import density

import sys
from time import time
from sklearn.grid_search import GridSearchCV
from sklearn.learning_curve import learning_curve


import pandas as pd
from statsmodels.graphics.mosaicplot import mosaic


matplotlib.style.use('ggplot')


Populating the interactive namespace from numpy and matplotlib


ImportError: No module named deprecation

### Exploring The Variables :
The first step in exploratory analysis is reading in the data and then exploring the variables. It is important to get a sense of how many variables and cases there are, the data types of the variables and the range of values they take on.

In [ ]:
# data uploded into the data frame  
loan_data_df  = pd.read_csv('LoansTrainingSet.csv')

#convert the loan status into number;
loan_data_df['flag'] = loan_data_df['Loan Status'].replace(['Charged Off' ,'Fully Paid'], 
                     [1, 0]) ;

# Rename All Columns :
loan_data =loan_data_df.rename(columns = {'Loan ID':'loan_id' ,\
                                          'Customer ID' :'customer_id' ,\
                                          'Loan Status' :'loan_status',\
                                          'Current Loan Amount' : 'current_loan_amount' ,\
                                          'Term' :'term',\
                                          'Credit Score':'credit_score' ,\
                                          'Years in current job' : 'years_in_current_job',\
                                          'Home Ownership' :'home_ownership' ,\
                                          'Annual Income' :'annual_income' ,\
                                          'Purpose' :'purpose',\
                                          'Monthly Debt' :'monthly_debt',\
                                          'Years of Credit History' :'years_of_credit_history' ,\
                                          'Months since last delinquent' :'months_since_last_delinquent',
                                          'Number of Open Accounts' : 'number_of_open_accounts' ,
                                          'Number of Credit Problems': 'number_of_credit_problems',
                                          'Current Credit Balance' : 'current_credit_balance' ,
                                          'Maximum Open Credit' :'maximum_open_credit',
                                          'Bankruptcies':'bankruptcies',
                                          'Tax Liens' :'tax_liens',
                                          'flag' :'defaulter_or_not'

                                           
                                           })





# Removing non numeric characters from the features columns
# This should be uncomment
loan_data['purpose'] =   map(lambda x: x.lower(), loan_data['purpose'])
#loan_data['monthly_debt'] = [x[1:] for x in loan_data['monthly_debt']]
#loan_data['monthly_debt'] = loan_data["monthly_debt"].str.replace(",", "").astype(float)
#loan_data = loan_data[loan_data['maximum_open_credit'] != '#VALUE!']


In [ ]:
print " Dimensions of Data "
print loan_data.shape           # Check dimensions
print "Data types : "
print loan_data.dtypes



Intially From the Source Data There are 20 Features and 256984 observations .Several of the column variables are encoded as numeric data types (ints and floats) but a few of them are encoded as "object". Let's check the head of the data to get a better sense of what the variables look like:

In [ ]:
print(loan_data.head(2))  # Check the first 5 rows

It appears we have a mixture of numeric columns and columns with text data. In data analysis, variables that split records into a fixed number of unique categories, such as Sex, are known as categorical variables. Pandas will attempt to interpret categorical variables as such when you load data, but you can specifically convert a variable to categorical if necessary, as we'll see later.

After getting a sense of the data's structure, it is a good idea to look at a statistical summary of the variables with df.describe():

In [ ]:
print( loan_data.describe() )

It is noticeable that non-numeric columns are dropped from the statistical summary provided by loan_data.describe().There are some non numerical values for this summary may not reflect the proper 
We can get a summary of the categorical variables by passing only those columns to describe():


In [ ]:
categorical = loan_data.dtypes[loan_data.dtypes == "object"].index
print(categorical)

loan_data[categorical].describe()

Although describe() gives a concise overview of each variable, it does not necessarily give us enough information to determine what each variable means.

In [ ]:
# Loan ID: A unique Identifier for the loan information.

# Customer ID: A unique identifier for the customer. Customers may have more than one loan.

# Loan Status: A categorical variable indicating if the loan was paid back or defaulted.

# Current Loan Amount: This is the loan amount that was either completely paid off, or the amount that was defaulted.

# Term: A categorical variable indicating if it is a short term or long term loan.

# Credit Score: A value between 0 and 800 indicating the riskiness of the borrowers credit history.

# Years in current job: A categorical variable indicating how many years the customer has been in their current job.

# Home Ownership: Categorical variable indicating home ownership. Values are "Rent", "Home Mortgage", and "Own". If the value is OWN, then the customer is a home owner with no mortgage

# Annual Income: The customer's annual income

# Purpose: A description of the purpose of the loan.

# Monthly Debt: The customer's monthly payment for their existing loans

# Years of Credit History: The years since the first entry in the customer’s credit history •
# Months since last delinquent: Months since the last loan delinquent payment

# Number of Open Accounts: The total number of open credit cards

#  Number of Credit Problems: The number of credit problems in the customer records.

# Current Credit Balance: The current total debt for the customer

#  Maximum Open Credit: The maximum credit limit for all credit sources.

# Bankruptcies: The number of bankruptcies

# Tax Liens: The number of tax liens.


After looking at the data for the first time, you should ask yourself a few questions:

   1. Do I need all of the variables?
   2. Should I transform any variables?
   3. Are there NA values, outliers or other strange values?
   4. Should I create new variables?

For the rest of this lesson we will address each of these questions in the context of this data set.


### Remove All variables that are necessary :
Getting rid of unnecessary variables is a good first step when dealing with any data set, since dropping variables reduces complexity and can make computation on the data faster. 

In [ ]:
del loan_data["customer_id"]
del loan_data["loan_id"]

#### Are there NA Values, Outliers or Other Strange Values?
Data sets are often littered with missing data, extreme data points called outliers and other strange values. Missing values, outliers and strange values can negatively affect statistical tests and models and may even cause certain functions to fail.
In Python, you can detect missing values with the pd.isnull() function:


1. Detecting Missing Values :
  let us first find out how many values are missing . Missing values shows the data discripency. Let us see whether we can find some detail insight about the data .

In [ ]:
loan_data.isnull().sum()

From the above chart we can find the  there are 61676 observations where credit score and annual income are not available . let us find how many of them are defaulter or not .


In [ ]:
missing_cr_score =   loan_data[pd.isnull(loan_data["credit_score"])] 
#missing['loan_status'] = missing['loan_status'].astype(str)
missing_cr_score= pd.Categorical(missing_cr_score['loan_status'] )

m  =  pd.DataFrame(missing_cr_score.describe())
m


 



In [ ]:
m.freqs.plot(kind ="bar" , color ="blue")

Approximately  31% loan has been charged off where credit score is not available . Let us explore annual_income and find how 
missing values impact there .

In [ ]:
missing_annual_income =   loan_data[pd.isnull(loan_data["annual_income"])] 
#missing['loan_status'] = missing['loan_status'].astype(str)
missing_annual_income = pd.Categorical(missing_annual_income['loan_status'] )

m  =  pd.DataFrame(missing_annual_income.describe())
m


Similar impact like credit score .so later we will try to explore relationship between annual_income and credit_score variables .

In [ ]:
missing =   loan_data[pd.isnull(loan_data["months_since_last_delinquent"])] 
#missing['loan_status'] = missing['loan_status'].astype(str)
missing = pd.Categorical(missing['loan_status'] )

m  =  pd.DataFrame(missing.describe())
m


In [ ]:
m.freqs.plot(kind ="bar" , color ="blue")

#### Categorical Variables in the Data Set :
There are four categorical Variables in the data set . Term of loans  , Purpose of loans , Home ownership and years in the current job . Among them though here years in current job is categorical variables but we will convert it to continuous as this will have significant impact on credit risk modeling .

There are two types of loan . short term and long term . 

In [ ]:
pd.options.display.float_format = '{:,.3f}'.format
def calculate_amount(r ) :
    return  r *1.0/ len(loan_data) 



ct = pd.crosstab(index=loan_data["term"], 
                          columns=loan_data["loan_status"] ).apply(lambda x : calculate_amount(x ) , axis = 1)


ct


In [ ]:
ct.plot(kind="bar", 
                 
                 stacked=False)

Term loan has more vulnerable approximately term loan given to 25% people among them 48% are defaulter which is 12 % of given data set . It seems Short term loan are more safe as it has 20% defaulter which is 26% of the given short term loan . It is provided that 32%  among provided data set are defaulter

In [ ]:
ct = pd.crosstab(index=loan_data["purpose"], 
                          columns=loan_data["loan_status"] ).apply(lambda x : calculate_amount(x ) , axis = 1)


ct

In [ ]:
ct.plot(kind="bar", 
                 
                 stacked=False)

Apporximately 80% loans are used for debt consolidation and 25% are defaulter . other loans are very nominal as home improvments amd misc. loans are in second and 3rd positions respectively.

In [ ]:
ct = pd.crosstab(index=loan_data["home_ownership"], 
                          columns=loan_data["loan_status"] ).apply(lambda x : calculate_amount(x ) , axis = 1)


ct

In [ ]:
ct.plot(kind="bar", 
                 
                 stacked=False)

Though most loans has been provided to them who has house mortgage . but most defaulter are those who live in house
with Rent . 15% are defaulter which  is more than 34 % of that loan which has been provided to people with rent house


We could leave 'years_in_current_job' as categorical data, but it shouldn't be treated as such or as ordinal data since the intervals are easy to determine. We can convert it into numerical data with a simple filter

In [ ]:
ct = pd.crosstab(index=loan_data["years_in_current_job"], 
                          columns=loan_data["loan_status"]  ).apply(lambda x : calculate_amount(x *100) , axis = 1)


ct

In [ ]:
ct.plot(kind="bar", 
                 
                 stacked=False)

From the Above graph it is found that most of the loans given to the employee who have more than 10+ years experience . Since It is a numerica variables we will convert it to numeric values .

In [ ]:
loan_data.replace('n/a', np.nan,inplace=True)
loan_data.years_in_current_job.fillna(value=0,inplace=True)
loan_data['years_in_current_job'].replace(to_replace='[^0-9]+', value='', inplace=True, regex=True)
loan_data['years_in_current_job'] = loan_data['years_in_current_job'].astype(int)
loan_data.years_in_current_job.value_counts()


#### Encoding categorical features :
We have 3 Categorical features .Purpose , Term , home_ownership ,years_in_current_job . We will convert them into distinct features using one hot encoding method and later we will drop the parent column.Before drop this let us understand what is the impact of this features into the data .

In [ ]:
def one_hot_encoding(df, colname ) :
    df[colname] =  df[colname].str.lower()
    df[colname] =  df[colname].str.replace(" " ,"_")
    df = pd.concat([df, pd.get_dummies(df[colname]).rename(columns=lambda x:   str(x))], axis=1) 
    df.drop(colname,axis =1 , inplace=True)
    
    return df

loan_data = one_hot_encoding(loan_data, 'purpose' ) ;
loan_data =  one_hot_encoding(loan_data, 'term' ) ;
loan_data = one_hot_encoding(loan_data, 'home_ownership' ) ;

del loan_data["loan_status"]



print loan_data.info()


#### Exploring Continuous Data :
Discrete data are, again, data with a finite or countably infinite number of possible outcomes. Continuous data, on the other hand, are data which come from an interval of possible outcomes.In our Data set examples of continuous variables include :
   - Current loan amount           
   - Credit score                  
   - Years in current job          
   - Annual income                 
   - Monthly debt                  
   - Years of credit history       
   - Months since last delinquent  
   - Number of open accounts       
   - Number of credit problems     
   - Current credit balance        
   - Maximum open credit      
   
In each of these examples, the resulting measurement comes from an interval of possible outcomes. and the measurement tool is often the restricting factor with continuous data. We'll summarize the data graphically using histograms, stem-and-leaf plots, and box plots.Here, we'll investigate how to summarize continuous data numerically using order statistics and various functions of order statistics.

### Imputing missing values :
Data sets are often littered with missing data, extreme data points called outliers and other strange values. Missing values, outliers and strange values can negatively affect statistical tests and models and may even cause certain functions to fail.

Detecting missing values is the easy part: it is far more difficult to decide how to handle them. In cases where we have a lot of data and only a few missing values, it might make sense to simply delete records with missing values present. On the other hand, if you have more than a handful of missing values, removing records with missing values could cause you to get rid of a lot of data. Missing values in categorical data are not particularly troubling because we can simply treat NA as an additional category. Missing values in numeric variables are more troublesome, since you can't just treat a missing value as number. 

Here are a few ways we could deal with them:
1. Replace the null values with 0s
2. Replace the null values with some central value like the mean or median
3. Impute values (estimate values using statistical/predictive modeling methods.).

This example shows that imputing the missing values can give better results than discarding the samples containing any missing value. Sometimes dropping rows or using marker values is more effective. Here we will tackle missing values with mean values of that column. So there is no missing Values .


In [ ]:

loan_data['annual_income'].fillna((loan_data['annual_income'].mean()), inplace=True)
loan_data['months_since_last_delinquent'].fillna((loan_data['months_since_last_delinquent'].mean()), inplace=True)
loan_data['credit_score'].fillna((loan_data['credit_score'].mean()), inplace=True)
loan_data['bankruptcies'].fillna((loan_data['bankruptcies'].median()), inplace=True)
loan_data['tax_liens'].fillna((loan_data['tax_liens'].median()), inplace=True)
loan_data['maximum_open_credit'].fillna((0), inplace=True)

loan_data.isnull().sum()

Now we will explore all continuous variables and explore all outliers .

In [ ]:
defaulter = loan_data[['current_loan_amount' ,'credit_score','years_in_current_job','annual_income','monthly_debt',\
                       'years_of_credit_history', \
                      'months_since_last_delinquent', 'number_of_open_accounts' ,'number_of_credit_problems' ,\
                       'current_credit_balance' ,  'maximum_open_credit']].loc[loan_data['defaulter_or_not'] == 1]

print defaulter.info()

non_defaulter = loan_data[['current_loan_amount' ,'credit_score','years_in_current_job','annual_income','monthly_debt'\
                           ,'years_of_credit_history', \
                      'months_since_last_delinquent', 'number_of_open_accounts' ,'number_of_credit_problems' ,\
                       'current_credit_balance' ,  'maximum_open_credit']].loc[loan_data['defaulter_or_not'] == 0]

print non_defaulter.info()

In [ ]:
def eda_continuous_var(colname) :
    f, axarr = plt.subplots(3, 3)
    f.set_size_inches(15, 9)
    ax1 = axarr[0, 0]
    ax1.set_title('Axis [0,0]')
    defaulter.hist(column=colname,        # Column to plot
                          # Plot size
                      color="red" ,
                      bins=50 ,ax =ax1) 

    non_defaulter.hist(column=colname,        # Column to plot
                          # Plot size
                      color="blue" ,
                      bins=100,ax = axarr[0, 1]) 

    loan_data.hist(column = colname,        # Column to plot
                          # Plot size
                      color="green" ,
                      bins=100,ax = axarr[0, 2]) 
    
    defaulter[colname].plot(kind ="box" , ax = axarr[1,0 ] , color ="red")
    non_defaulter[colname].plot(kind ="box" , ax = axarr[1,1] , color ="blue")
    loan_data[colname].plot(kind ="box" , ax = axarr[1,2 ] , color ="green")
    
    plt.setp([a.get_xticklabels() for a in axarr[0, :]], visible=False)
    plt.setp([a.get_yticklabels() for a in axarr[:, 1]], visible=False)


    plt.show()

eda_continuous_var('current_loan_amount') 

### Association Between  Variables :
Here we will learn how to statistically test for a relationship between two categorical variables. Categorical variables are related when the category that a subject is classified as for one variable "influences" the category that a subject is classified as in another variable. In other words, the category one is classified as for one variable "depends" on the category one is classified as in another variable. The chi-square (χ2χ2) test of independence will be used to test for this relationship.

#### Chi-Square Test of Independence :
The chi-square (χ2) test of independence is used to test for a statistically significant relationship between two categorical variables. This is an inferential test that uses data from a sample to make conclusions about the relationship between categorical variables in the population.

The chi-square distribution is a special type of right skewed distribution.  Like the t distribution, the chi-square distribution varies depending on the degrees of freedom. 

An observed relationship will be called statistically significant when the p-value for a chi-square test is less than αα (typically α=.05α=.05). In this case, if we reject the null hypothesis, then we generalize that there is a relationship in the population.

### Apply Machine Learning Model to Data Set :
To predict the class of the loan we will use supervised model for classification .our goal is to predict class level which is a choice of predefined list of possiblities . Our goal is to make accuratepredictions for new, never-before-seen data. Supervised learning oftenrequires human effort to build the training set, but afterwardautomates and often speeds up an otherwise laborious or infeasible task.For now we will go through the algorithm themselves by first .

#### k-Nearest Neighbours :
The k-NN algorithm is arguably the simplest machinelearning algorithm. Building the model consists only of storing the training dataset. To make a prediction for a new data point, the  algorithm finds the closest data points in the training dataset—its “nearest neighbors.”
##### How do we choose the factor K?
we can see that the boundary becomes smoother with increasing value of K. With K increasing to infinity it finally becomes all blue or all red depending on the total majority.  The training error rate and the validation error rate are two parameters we need to access on different K-value. Following is the curve for the training error rate with varying value of K :

In [ ]:
features = list(loan_data.columns)
features.remove('defaulter_or_not');

target =  'defaulter_or_not'


X =  loan_data[list(features)].values
y = loan_data[target].values

X_samples = X[1:50000]
y_samples = y[1:50000]



train_X, test_X, train_Y, test_Y =  train_test_split(X_samples, y_samples, test_size= .5, random_state=66)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

training_accuracy = []
test_accuracy = []# try n_neighbors from 1 to 10
neighbors_settings = range(1, 11)
for n_neighbors in neighbors_settings:    # build the model   
    clf = KNeighborsClassifier(n_neighbors=n_neighbors)    
    clf.fit(train_X, train_Y)    # record training set accuracy
    training_accuracy.append(clf.score(train_X, train_Y))    # record generalization accuracy   
    test_accuracy.append(clf.score(test_X, test_Y))

plt.plot(neighbors_settings, training_accuracy, label="training accuracy")
plt.plot(neighbors_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("n_neighbors")
plt.legend()
plt.show()



From the above diagram we can find that for K=1 we get training accuracy 100 % but test accuracy less than 75%. if k=8 to 10 we get less difference between test and train data . for our modelling we will use k=10

##### STRENGTHS, WEAKNESSES, AND PARAMETERS of kNN :
In principle, there are two important parameters to the KNeighbors classifier: 
1. the number of neighbors 
2. how we measure distance between data points. In practice, using a small number of neighbors like three or five often works well, but you should certainly adjust this parameter. 

There are several advantages of Knn :
The model is very easy to understand, and often gives reasonable performance without a lot of adjustments .Using this algorithm is a good baseline method to try before considering mIore advanced techniques.
On the other hand though it is fast but for larger training set prediction may be slow .It does not perform well when data set have many features . In these later method we will try to reduce or overcome the limitation of Knn

#### LINEAR MODELS FOR CLASSIFICATION :
The two most common linear classification algorithms are logistic regression, implemented in linear_model.LogisticRegression, and linear support vector machines (linear SVMs), implemented in svm.LinearSVC (SVC stands for support vector classifier).

The two models come up with similar decision boundaries. Note that both misclassify two of the points. By default, both models apply an L2 regularization, in the same way that Ridge does for regression.

For LogisticRegression and LinearSVC the trade-off parameter that determines the strength of the regularization is called C, and higher values of C correspond to less regularization. In other words, when you use a high value for the parameter C, LogisticRegression and LinearSVC try to fit the training set as best as possible, while with low values of the parameter C, the models put more emphasis on finding a coefficient vector (w) that is close to zero.

There is another interesting aspect of how the parameter C acts. Using low values of C will cause the algorithms to try to adjust to the “majority” of data points, while using a higher value of C stresses the importance that each individual data point be classified correctly. 


In [ ]:

training_accuracy = []
test_accuracy = []# try n_neighbors from 1 to 10
c_settings = range(1,100 ,10)
for c in c_settings:    # build the model   
    clf = LogisticRegression(C=c)    
    clf.fit(train_X, train_Y)    # record training set accuracy
    training_accuracy.append(clf.score(train_X, train_Y))    # record generalization accuracy   
    test_accuracy.append(clf.score(test_X, test_Y))

plt.plot(c_settings, training_accuracy, label="training accuracy")
plt.plot(c_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("C = regularization Parameter")
plt.legend()
plt.show()



For this Logistic regression we are plotting and finding now much variation . though training accuracy is less then test accuracy .

#### Naive Bayes Classifiers :
Naive Bayes classifiers are a family of classifiers that are quite similar to the linear models discussed in the previous section. However, they tend to be even faster in training. The price paid for this efficiency is that naive Bayes models often provide generalization performance that is slightly worse than that of linear classifiers like LogisticRegression and LinearSVC.

The reason that naive Bayes models are so efficient is that they learn parameters by looking at each feature individually and collect simple per-class statistics from each feature. 

##### STRENGTHS, WEAKNESSES, AND PARAMETERS :
MultinomialNB and BernoulliNB have a single parameter, alpha, which controls model complexity. The way alpha works is that the algorithm adds to the data alpha many virtual data points that have positive values for all the features. This results in a “smoothing” of the statistics. A large alpha means more smoothing, resulting in less complex models. 

The naive Bayes models share many of the strengths and weaknesses of the linear models. They are very fast to train and to predict, and the training procedure is easy to understand. The models work very well with high-dimensional sparse data and are relatively robust to the parameters. Naive Bayes models are great baseline models and are often used on very large datasets, where training even a linear model might take too long.






#### Decision Trees :
   Decision tree is a type of supervised learning algorithm (having a pre-defined target variable) that is mostly used in classification problems. It works for both categorical and continuous input and output variables. In this technique, we split the population or sample into two or more homogeneous sets (or sub-populations) based on most significant splitter / differentiator in input variables.
   
How does a tree decide where to split : 
   The decision of making strategic splits heavily affects a tree’s accuracy. Decision trees use multiple algorithms to decide to split a node in two or more sub-nodes. The creation of sub-nodes increases the homogeneity of resultant sub-nodes. In other words, we can say that purity of the node increases with respect to the target variable. Decision tree splits the nodes on all available variables and then selects the split which results in most homogeneous sub-nodes.  
   
   Gini Index :
   if we select two items from a population at random then they must be of same class and probability for this is 1 if population is pure.
   
   Chi-Square :
   It is an algorithm to find out the statistical significance between the differences between sub-nodes and parent node. We measure it by sum of squares of standardized differences between observed and expected frequencies of target variable.
   
   Information Gain:
   Look at the image below and think which node can be described easily. I am sure, your answer is C because it requires less information as all values are similar.
   
   Reduction in Variance :
   Till now, we have discussed the algorithms for categorical target variable. Reduction in variance is an algorithm used for continuous target variables (regression problems). This algorithm uses the standard formula of variance to choose the best split. The split with lower variance is selected as the criteria to split the population:
   
   \begin{equation*}
   varience = \frac{\displaystyle\sum_{i=1}^{n}(x_i - \mu)^2} {n}
   \end{equation*}
   

In [ ]:

training_accuracy = []
test_accuracy = []# try n_neighbors from 1 to 10
max_depth_settings = range(1,100 ,2)
for c in max_depth_settings:    # build the model   
    clf = DecisionTreeClassifier(random_state=0 , max_depth=c ,max_leaf_nodes = 20 ,criterion =  "entropy" )   
    clf.fit(train_X, train_Y)    # record training set accuracy
    training_accuracy.append(clf.score(train_X, train_Y))    # record generalization accuracy   
    test_accuracy.append(clf.score(test_X, test_Y))

plt.plot(max_depth_settings, training_accuracy, label="training accuracy")
plt.plot(max_depth_settings, test_accuracy, label="test accuracy")
plt.ylabel("Accuracy")
plt.xlabel("C = Maximum Depth")
plt.legend()
plt.show()


#### STRENGTHS, WEAKNESSES, AND PARAMETERS :
As discussed earlier, the parameters that control model complexity in decision trees are the pre-pruning parameters that stop the building of the tree before it is fully developed. Usually, picking one of the pre-pruning strategies—setting either max_depth, max_leaf_nodes, or min_samples_leaf—is sufficient to prevent overfitting.

Advantages includes :
the resulting model can easily be visualized and understood by nonexperts (at least for smaller trees), and the algorithms are completely invariant to scaling of the data. As each feature is processed separately, and the possible splits of the data don’t depend on scaling, no preprocessing like normalization or standardization of features is needed for decision tree algorithms. In particular, decision trees work well when you have features that are on completely different scales, or a mix of binary and continuous features.

The main downside of decision trees is that even with the use of pre-pruning, they tend to overfit and provide poor generalization performance. Therefore, in most applications, the ensemble methods we discuss next are usually used in place of a single decision tree.

#### Random Forest Model :
Bagging is a technique used to reduce the variance of our predictions by combining the result of multiple classifiers modeled on different sub-samples of the same data set. The following figure will make it clearer .
The steps followed in bagging are:

1. Create Multiple DataSets:
Sampling is done with replacement on the original data and new datasets are formed.The new data sets can have a fraction of the columns as well as rows, which are generally hyper-parameters in a bagging model.
Taking row and column fractions less than 1 helps in making robust models, less prone to overfitting
2. Build Multiple Classifiers:
Classifiers are built on each data set.
Generally the same classifier is modeled on each data set and predictions are made.
3. Combine Classifiers:
The predictions of all the classifiers are combined using a mean, median or mode value depending on the problem at hand.
The combined values are generally more robust than a single model.

#### STRENGTHS, WEAKNESSES, AND PARAMETERS :

This algorithm can solve both type of problems i.e. classification and regression and does a decent estimation at both fronts.
One of benefits of Random forest which excites me most is, the power of handle large data set with higher dimensionality. It can handle thousands of input variables and identify most significant variables so it is considered as one of the dimensionality reduction methods. Further, the model outputs Importance of variable, which can be a very handy feature (on some random data set).

#### Extreme Gradient Boosting (xgboost) :

Extreme Gradient Boosting (xgboost) is similar to gradient boosting framework but more efficient. It has both linear model solver and tree learning algorithms. So, what makes it fast is its capacity to do parallel computation on a single machine.

This makes xgboost at least 10 times faster than existing gradient boosting implementations. It supports various objective functions, including regression, classification and ranking.

Since it is very high in predictive power but relatively slow with implementation, “xgboost” becomes an ideal fit for many competitions. It also has additional features for doing cross validation and finding
Gradient Boosted Regression Trees :


Strengths, weaknesses, and parameters
Gradient boosted decision trees are among the most powerful and widely used models for supervised learning. Their main drawback is that they require careful tuning of the parameters and may take a long time to train. Similarly to other tree-based models, the algorithm works well without scaling and on a mixture of binary and continuous features. As with other tree-based models, it also often does not work well on high-dimensional sparse data.

The main parameters of gradient boosted tree models are the number of trees, n_estimators, and the learning_rate, which controls the degree to which each tree is allowed to correct the mistakes of the previous trees. These two parameters are highly interconnected, as a lower learning_rate means that more trees are needed to build a model of similar complexity. In contrast to random forests, where a higher n_estimators value is always better, increasing n_estimators in gradient boosting leads to a more complex model, which may lead to overfitting. A common practice is to fit n_estimators depending on the time and memory budget, and then search over different learning_rates.

#### Multi-layer Perceptron (MLP)  :

Multi-layer Perceptron (MLP) is a supervised learning algorithm that learns a function by training on a dataset, where m is the number of dimensions for input and o is the number of dimensions for output. Given a set of features X = {x_1, x_2, ..., x_m} and a target y, it can learn a non-linear function approximator for either classification or regression. It is different from logistic regression, in that between the input and the output layer, there can be one or more non-linear layers, called hidden layers. 

The advantages of Multi-layer Perceptron are:
   1. Capability to learn non-linear models.
   2. Capability to learn models in real-time (on-line learning) using partial_fit.

The disadvantages of Multi-layer Perceptron (MLP) include:
1. MLP with hidden layers have a non-convex loss function where there exists more than one local minimum. Therefore different random weight initializations can lead to different validation accuracy.
2. MLP requires tuning a number of hyperparameters such as the number of hidden neurons, layers, and iterations.
3. MLP is sensitive to feature scaling.

#### Evaluation : 
In classification problems, we use two types of algorithms (dependent on the kind of output it creates):

Class output : Algorithms like SVM and KNN create a class output. For instance, in a binary classification problem, the outputs will be either 0 or 1. However, today we have algorithms which can convert these class outputs to probability. But these algorithms are not well accepted by the statistics community.

Probability output : Algorithms like Logistic Regression, Random Forest, Gradient Boosting, Adaboost etc. give probability outputs. Converting probability outputs to class output is just a matter of creating a threshold probability.

#### Confusion Matrix

A confusion matrix is an N X N matrix, where N is the number of classes being predicted. For the problem in hand, we have N=2, and hence we get a 2 X 2 matrix. Here are a few definitions, you need to remember for a confusion matrix :

Accuracy : the proportion of the total number of predictions that were correct.
Positive Predictive Value or Precision : the proportion of positive cases that were correctly identified.
Negative Predictive Value : the proportion of negative cases that were correctly identified.
Sensitivity or Recall : the proportion of actual positive cases which are correctly identified.
Specificity : the proportion of actual negative cases which are correctly identified.

In [ ]:
# Benchmark classifiers
def benchmark(clf, name ):

    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    if name == "XGboost Classifier" :
        clf.fit(train_X, train_Y ,eval_metric='auc' )
    else :
        clf.fit(train_X, train_Y)
            
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)
    print()
    t0 = time()
    y_pred = clf.predict(test_X)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    
    score = metrics.accuracy_score(test_Y, y_pred)
    print("accuracy:   %0.3f" % score)
    if hasattr(clf, 'coef_'):
        print("dimensionality: %d" % clf.coef_.shape[1])
        print("density: %f" % density(clf.coef_))
    roc = roc_auc_score(test_Y, y_pred)
    print ("AUC Score (Roc): %0.3f" % roc )
    recall = recall_score(test_Y, y_pred)

    print ("recall score :%0.3f " % recall )
    f1 = f1_score(test_Y, y_pred) 
    print ("F1 score :%0.3f"  %  f1 )
    precision = precision_score(test_Y, y_pred)
    print (  "Precision score : %0.3f " % precision )
    
    clf_descr = str(clf).split('(')[0]
    return clf_descr,score , train_time ,test_time , roc ,  recall , f1, precision  

In [ ]:

results = []
for clf, name in ( (LogisticRegression() ,"Logistic Regression "),
        (RidgeClassifier(tol=1e-2, solver="lsqr"), "Ridge Classifier"),
        (DecisionTreeClassifier(min_samples_split =5 ,max_depth= 10 ),
         p   ),
        
        (RandomForestClassifier(n_estimators=15), "Random forest"),
                  ( XGBClassifier(
                     learning_rate =0.1,
                     n_estimators=50,
                     max_depth=5,
                     min_child_weight=1,
                     gamma=0,
                     subsample=0.8,
                     colsample_bytree=0.8,
                     objective= 'binary:logistic',
                     nthread=4,
                     scale_pos_weight=1,
                     seed=27) ,"XGboost Classifier" )
                 
                 
                 
                 
                 ):
    print('=' * 80)
    print(name)
    results.append(benchmark(clf ,name ))
    
    


indices = np.arange(len(results))
#print (results)

results = [[x[i] for x in results] for i in range(8 )]


clf_names, score, training_time, test_time,  roc ,  recall , f1, precision  = results
training_time = np.array(training_time) / np.max(training_time)
test_time = np.array(test_time) / np.max(test_time)


plt.figure(figsize=(12, 8))
plt.title("Score")
plt.barh(indices, score, .1, label="score", color='r')
plt.barh(indices + .1, training_time, .1, label="training time", color='g')
plt.barh(indices + .2, test_time, .1, label="test time", color='b')
plt.barh(indices + .3, roc , .1, label="ROC", color='m')
plt.barh(indices + .4, recall , .1, label="recall", color='y')
plt.barh(indices + .5, f1 , .1, label="f1", color='k')
plt.barh(indices + .6, f1 , .1, label="precision", color='c')


plt.yticks(())
plt.legend(loc='best')
plt.subplots_adjust(left=.
                    25)
plt.subplots_adjust(top=.95)
plt.subplots_adjust(bottom=.05)

for i, c in zip(indices, clf_names):
    plt.text(-.3, i, c)

plt.show()
    
    



#### Area Under the ROC curve (AUC – ROC) :
The ROC curve is the plot between sensitivity and (1- specificity). (1- specificity) is also known as false positive rate and sensitivity is also known as True Positive rate. Following is the ROC curve for the case in hand.

he sensitivity at this threshold is 99.6% and the (1-specificity) is ~60%. This coordinate becomes on point in our ROC curve. To bring this curve down to a single number, we find the area under this curve (AUC).

Note that the area of entire square is 1*1 = 1. Hence AUC itself is the ratio under the curve and the total area. For the case in hand, we get AUC ROC as 96.4%. Following are a few thumb rules:

.90-1 = excellent (A)
.80-.90 = good (B)
.70-.80 = fair (C)
.60-.70 = poor (D)
.50-.60 = fail (F)
We see that we fall under the excellent band for the current model. But this might simply be over-fitting. In such cases it becomes very important to to in-time and out-of-time validations.
